# Interoperability with C

Julia is interoperable with C, provided it is available as a dynamic Library. In the example below (borrowed from Steven G. Johnson's [lecture notes](https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)), the `sum` function is implemented in C, and a Julia wrapper is created.

In [1]:
using Libdl

C_code = """
#include <stdio.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -fPIC -O3 -ffast-math -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [2]:
a = rand(100)
c_sum(a), sum(a)
@time c_sum(a)
@time sum(a)

  0.000004 seconds (1 allocation: 16 bytes)
  0.000004 seconds (1 allocation: 16 bytes)


48.673108263025085

This is how the low-level linear algebra is performed in Julia, *i.e.* `axpy!`, `axpby!` just wrap Blas/MKL... libraries, whereas the standard LinearAlgebra module simply wraps [SuiteSparse](https://people.engr.tamu.edu/davis/suitesparse.html).

It's actually pretty straightforward to wrap a C/Fortran library, build binaries for various platforms, write a wrapper and ultimately make it as simple as
```
pkg> add MyFancyLibrary
```
to users. A typically workflow is

1. Start from a public repository when the C code is hosted with a Makefile/CMake... build system,
1. Use [BinaryBuilder.jl](https://github.com/JuliaPackaging/BinaryBuilder.jl) to write a script for the build process and automatically generate and register a Julia package (`.jll`) which generate binaries for different platform,
1. Write a Julia package with a user-friendly ("Julia") interface.

In the same way that calling C from Julia, it's also possible to pass Julia functions as callback functions to C functions, etc...

# Metaprogramming

(This section is heavily based on a notebook by Jeff Bezanson available [here](https://github.com/mitmath/18S096/blob/master/lectures/lecture7/Metaprogramming.ipynb).)

A "meta" program is a program that manipulates programs. While Julia can generate specialized code for you (and the compiler is usually smart enough to infer a lot of information, *e.g.* constant propagation), sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code, that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

## Symbols and expressions

Building block to represent programs include unevaluated symbols and expressions. A symbol is defined as

In [3]:
:x

:x

In [4]:
typeof(:x)

Symbol

Symbols can be evaluated to retrieve the value they are associated with.

In [5]:
a = -1

-1

In [6]:
eval(:a)

-1

Operators and function names are also symbols.

In [7]:
sym = :sin

:sin

In [8]:
eval(sym)

sin (generic function with 14 methods)

Symbols can be combined to form expressions (`Expr`).

In [9]:
expr = :(sin(a))

:(sin(a))

In [10]:
dump(expr)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Symbol a


Expressions can in turn be combined with symbols to form more complex expressions. This can be done explicitly

In [11]:
Expr(:call, :sin, Expr(:call, :*, :π, :x))

:(sin(π * x))

or using interpolation (similarly to string interpolation in the second notebook).

In [12]:
expr = :(π * x)
:(sin($expr))

:(sin(π * x))

# Macros

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](https://docs.julialang.org/en/v1/manual/metaprogramming/) puts it like this: macros map a tuple of argument *expressions* to a returned *expression*.

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language,
- to rearrange or delay evaluation of code,
- to eliminate boilerplate (repetitive) code,
- to automatically generate complex code that would be painful by hand,
- to unroll loops for efficiency,
- to inline code for efficiency.

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [13]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [14]:
x = 0
@twice println(x)

0
0


### Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:
$$
p \colon x \mapsto a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0
$$
may be evaluated efficiently as
$$
p \colon x \mapsto a_0 + x \left (a_1 + \cdots x \left ( a_{n-2} + \cdots + x \left (a_{n-1} + x a_n \right ) \cdots \right ) \right )
$$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [15]:
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )  # actually uses `muladd`
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

This is called as follows: to evaluate the polynomial $p \colon x \mapsto 2 + 3x + 4x^2$ at $x=3$,

In [16]:
x = 3
@horner(x, 2, 3, 4, 5)

182

To see what the macro does to this call, use `macroexpand`

In [17]:
macroexpand(Main, :(@horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

quote
    var"#44#t" = x
    2 + var"#44#t" * (3 + var"#44#t" * (4 + var"#44#t" * (5 + var"#44#t" * (6 + var"#44#t" * (7 + var"#44#t" * (8 + var"#44#t" * (9 + var"#44#t" * 10)))))))
end

# `@generated` functions

Another form of metaprogramming available in Julia. The idea is to generate code based on types instead of the input expressions.

The Julia parser and compiler have become some good over the years that it's actually rarely used anymore.

An example we came across this week was the following: imagine a tuple of length `N` with `M ≤ N` elements are `<:AbstractVector`, whose elements we wish to access with a tuple of `<:Integer`s of length `M`.

In [18]:
_getindex(::Type{<:Number}, arg, _) = false, :(getindex($arg))
_getindex(::Type{<:AbstractVector}, arg, ind) = true, :(getindex($arg, $ind))

@generated function Base.getindex(args::Tuple, inds::Tuple)
    j, expr = 1, Expr(:tuple)

    for (i, T) in enumerate(fieldtypes(args))
        next, arg = _getindex(T, :(args[$i]), :(inds[$j]))
        j += next
        push!(expr.args, arg)
    end

    expr
end         

In [19]:
args, inds = (1, 2:3, 4:5), (1, 2)

((1, 2:3, 4:5), (1, 2))

In [20]:
args[inds]

(1, 2, 5)

In [21]:
@code_warntype args[inds]

MethodInstance for getindex(::Tuple{Int64, UnitRange{Int64}, UnitRange{Int64}}, ::Tuple{Int64, Int64})
  from getindex(args::Tuple, inds::Tuple) in Main at In[18]:4
Arguments
  #self#::Core.Const(getindex)
  args::Tuple{Int64, UnitRange{Int64}, UnitRange{Int64}}
  inds::Tuple{Int64, Int64}
Body::Tuple{Int64, Int64, Int64}
1 ─ %1 = Base.getindex(args, 1)::Int64
│   %2 = Main.getindex(%1)::Int64
│   %3 = Base.getindex(args, 2)::UnitRange{Int64}
│   %4 = Base.getindex(inds, 1)::Int64
│   %5 = Main.getindex(%3, %4)::Int64
│   %6 = Base.getindex(args, 3)::UnitRange{Int64}
│   %7 = Base.getindex(inds, 2)::Int64
│   %8 = Main.getindex(%6, %7)::Int64
│   %9 = Core.tuple(%2, %5, %8)::Tuple{Int64, Int64, Int64}
└──      return %9



# Type instability

Although this was covered quickly, we introduced the concept of parameterized types to avoid

In [22]:
struct MyBadType
    a
    b
end

which is equivalent to
```
struct MyBadType
    a::Any
    b::Any
end
```
and instead use

In [23]:
struct MyGoodType{A,B}
    a::A
    b::B
end

In [24]:
Base.sum(this::Union{MyBadType,MyGoodType}) = this.a + this.b

In [25]:
using BenchmarkTools

In [26]:
n = 1_000_000
bad = [MyBadType(rand(Float64), rand(Float64)) for i in 1:n]
good = [MyGoodType(rand(Float64), rand(Float64)) for i in 1:n];

In [27]:
sum(sum, bad); @time sum(sum, bad)

  0.095506 seconds (2.00 M allocations: 30.518 MiB, 57.30% gc time)


1.000240516691319e6

In [28]:
sum(sum, good); @time sum(sum, good)

  0.000846 seconds (1 allocation: 16 bytes)


999734.8910217271

The problem can easily be detected using the `@code_warntype` macro.

In [29]:
@code_warntype sum(first(bad))

MethodInstance for sum(::MyBadType)
  from sum(this::Union{MyBadType, MyGoodType}) in Main at In[24]:1
Arguments
  #self#::Core.Const(sum)
  this::MyBadType
Body::Any
1 ─ %1 = Base.getproperty(this, :a)::Any
│   %2 = Base.getproperty(this, :b)::Any
│   %3 = (%1 + %2)::Any
└──      return %3



The compiler isn't able to infer the return type at compile type.

In [30]:
@code_warntype sum(first(good))

MethodInstance for sum(::MyGoodType{Float64, Float64})
  from sum(this::Union{MyBadType, MyGoodType}) in Main at In[24]:1
Arguments
  #self#::Core.Const(sum)
  this::MyGoodType{Float64, Float64}
Body::Float64
1 ─ %1 = Base.getproperty(this, :a)::Float64
│   %2 = Base.getproperty(this, :b)::Float64
│   %3 = (%1 + %2)::Float64
└──      return %3



The issue can sneak in very easily.

In [31]:
function mysum(a)
    s = 0
    for x in a
        s += x
    end
    return s
end

mysum (generic function with 1 method)

In [32]:
a = rand(1:1:10, 1_000)
@code_warntype mysum(a)

MethodInstance for mysum(::Vector{Int64})
  from mysum(a) in Main at In[31]:1
Arguments
  #self#::Core.Const(mysum)
  a::Vector{Int64}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  s::Int64
  x::Int64
Body::Int64
1 ─       (s = 0)
│   %2  = a::Vector{Int64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (s = s + x)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s

